<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_McGill_NLP_medal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datacamp.com/tutorial/mistral-7b-tutorial

# DEPENDENCIES

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install ninja packaging --quiet


# Uncomment only if you're using A100 GPU or L4 GPU
!pip install flash-attn --no-build-isolation

!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import colab_env

In [ ]:
import torch
import tensorboard
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [ ]:
# set device
device = 'cuda'

In [ ]:
torch.__version__

'2.3.0+cu121'

In [ ]:
#!apt-get update && apt-get install -y cuda-11-0

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun Jun 16 02:16:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0       

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

# Data loading


https://huggingface.co/datasets/sakharamg/AviationQA

In [ ]:
#! pip install datasets  --quiet

In [ ]:
#from datasets import load_dataset
#print("Preprocessing dataset Emotion")
#dataset = load_dataset("McGill-NLP/medal",trust_remote_code=True)

#%mkdir -p /content/gdrive/MyDrive/datasets/McGill-NLP
#%cd /content/gdrive/MyDrive/datasets/McGill-NLP

# save datasets to disk
#dataset["train"].to_json("train_dataset.json", orient="records")
#dataset["validation"].to_json("validation_dataset.json", orient="records")
#dataset["test"].to_json("test_dataset.json", orient="records")

In [ ]:
#print(dataset)

In [10]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
print(dataset)

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 3000000
})


In [12]:
print(dataset[6])

{'abstract_id': 14445409, 'text': 'to quantify the amount of CL NO harvested in modified RND', 'location': [10], 'label': ['radical neck dissection']}


In [13]:
print(dataset[6]['text'])

to quantify the amount of CL NO harvested in modified RND


In [14]:
print(dataset[6]['label'])

['radical neck dissection']


In [15]:
train_question=dataset['text']
train_answer=dataset['label']

In [16]:
!rm -rf /content/medal-qa.txt
filename='/content/medal-qa.txt'

import re

# python object to be appended
for n in range(10000):
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    string = train_answer[n]
    #print(string)
    #result = string.replace("\nRef", "-Ref")
    str_question=train_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\[", "", str(string))
    answer = re.sub("\]", "", str(answer))

    #print(answer)
    if len(answer)==0:
        answer='Not possible to get or use'
    text='<s>[INST] %s [/INST] %s </s>'%(question,answer)
    #print(text)
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [17]:
train_question[6]

'to quantify the amount of CL NO harvested in modified RND'

In [18]:
train_answer[6]

['radical neck dissection']

In [19]:
text0 = load_dataset("text", data_files="/content/medal-qa.txt", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
nrec=10000

dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
dataset_final_text=text0[0:nrec]['text']

In [21]:
dataset[6]

{'abstract_id': 14445409,
 'text': 'to quantify the amount of CL NO harvested in modified RND',
 'location': [10],
 'label': ['radical neck dissection']}

In [22]:
text0[6]

{'text': "<s>[INST] to quantify the amount of CL NO harvested in modified RND [/INST] 'radical neck dissection' </s>"}

In [23]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

In [24]:
datasetF

,Question,Answer,text
0,velvet antlers vas are commonly used in tradit...,[transverse aortic constriction],<s>[INST] velvet antlers vas are commonly used...
1,the clinical features of our cases demonstrate...,[hodgkins lymphoma],<s>[INST] the clinical features of our cases d...
2,ceftobiprole bpr is an investigational cephalo...,[methicillinsusceptible s aureus],<s>[INST] ceftobiprole bpr is an investigation...
3,we have taken a basic biologic RPA to elucidat...,[parathyroid hormonerelated protein],<s>[INST] we have taken a basic biologic RPA t...
4,lipoperoxidationderived aldehydes for example ...,[lipoperoxidation],<s>[INST] lipoperoxidationderived aldehydes fo...
...,...,...,...
9995,the regulation of the number of extrajunctiona...,[intracellular cl],<s>[INST] the regulation of the number of extr...
9996,a highresolution structure of the HPr hpr from...,[histidinecontaining phosphocarrier protein],<s>[INST] a highresolution structure of the HP...
9997,the short term metabolic effects of the in viv...,[hexose monophosphate pathway],<s>[INST] the short term metabolic effects of ...
9998,we examined the effect of nnntrimethylsphingos...,[nndimethylsphingosine],<s>[INST] we examined the effect of nnntrimeth...


In [ ]:
datasetF['Question'][6]

'to quantify the amount of CL NO harvested in modified RND'

In [ ]:
datasetF['Answer'][6]

['radical neck dissection']

In [ ]:
datasetF['text'][6]

"<s>[INST] to quantify the amount of CL NO harvested in modified RND [/INST] 'radical neck dissection' </s>"

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi         9Gi       2.1Gi        13Mi        71Gi        72Gi
Swap:             0B          0B          0B


# Loading the Mistral 7B model

In [ ]:
!pip install mistral_inference -q

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install accelerate -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #10 june 2024,

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
#model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
   

Inference

Build an inference pipeline with tokenizer and the model.

In [ ]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt0="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

prompt = f"Instruct: Find THE answer for the following  question.\n{prompt0}\nOutput:\n"
#prompt = f"Instruct: Find only the answer for the following  question.\n{prompt0}"

#prompt="What are some interesting sites to visit in the Bay Area?"
#prompt="What is the capital of russia?"
outputs = pipe(prompt0, max_new_tokens=128, temperature=0.9,do_sample=True,top_k=30, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt0)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt0):].strip()}")


Question: What is the capital of russia?

Generated Answer:
Moscow


In [ ]:
print(outputs[0]['generated_text'])

What is the capital of russia?
Moscow


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
datasetF

,Question,Answer,text
0,velvet antlers vas are commonly used in tradit...,[transverse aortic constriction],<s>[INST] velvet antlers vas are commonly used...
1,the clinical features of our cases demonstrate...,[hodgkins lymphoma],<s>[INST] the clinical features of our cases d...
2,ceftobiprole bpr is an investigational cephalo...,[methicillinsusceptible s aureus],<s>[INST] ceftobiprole bpr is an investigation...
3,we have taken a basic biologic RPA to elucidat...,[parathyroid hormonerelated protein],<s>[INST] we have taken a basic biologic RPA t...
4,lipoperoxidationderived aldehydes for example ...,[lipoperoxidation],<s>[INST] lipoperoxidationderived aldehydes fo...
...,...,...,...
9995,the regulation of the number of extrajunctiona...,[intracellular cl],<s>[INST] the regulation of the number of extr...
9996,a highresolution structure of the HPr hpr from...,[histidinecontaining phosphocarrier protein],<s>[INST] a highresolution structure of the HP...
9997,the short term metabolic effects of the in viv...,[hexose monophosphate pathway],<s>[INST] the short term metabolic effects of ...
9998,we examined the effect of nnntrimethylsphingos...,[nndimethylsphingosine],<s>[INST] we examined the effect of nnntrimeth...


In [ ]:
index=6


prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

prompt = datasetF['text'][index]
print(prompt)

to quantify the amount of CL NO harvested in modified RND
['radical neck dissection']
<s>[INST] to quantify the amount of CL NO harvested in modified RND [/INST] 'radical neck dissection' </s>


MODEL GENERATION - ZERO SHOT

In [ ]:
index=6

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}"

res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=128, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
to quantify the amount of CL NO harvested in modified RND
---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
['radical neck dissection']

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
to quantify the amount of CL NO harvested in modified RND cells, we need to determine the number of CL NO cells that are present in the modified RND cells.

To determine the number of CL NO cells in the modified RND cells, we need to use a technique called flow cytometry. Flow cytometry is a technique that uses a laser to scatter light through a sample of cells. The amount of light scattered by each cell is proportional to its size and granularity. By analyzing the light scattered by th

In [ ]:
print(len(datasetF))

10000


In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
   

# Adding the adopter to the layer

In [ ]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# Hyperparmeters

In [ ]:
!pip install accelerate -U

https://github.com/huggingface/transformers/issues/26969

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/Mistral-7B-v0.1_McGill-MEDAL",

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=40,                   # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    #2
    gradient_accumulation_steps=6,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer

    #trainer = Trainer(model=model, args=training_args, train_dataset=ds, optimizers=(adam_bnb_optim, None))
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    #learning_rate=1.41e-5,
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [ ]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['Question', 'Answer', 'text'], dtype='object')


In [ ]:
print(datasetF['Answer'][6])

['radical neck dissection']


# Supervised fine-tuning (SFT)  parameters

In [ ]:
!pip install trl -q

https://pytorch.org/docs/stable/checkpoint.html#torch.utils.checkpoint.checkpoint

The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

The use_reentrant=False option is generally recommended because:

1.- Performance: It is often faster, especially when there are many re-entrant functions.

2.- Safety: It avoids potential issues with re-entrant functions that might modify global states during the backward pass.

The use_reentrant=True option exists for backward compatibility and situations where modifying the code is difficult. However, it can be less efficient and potentially lead to unexpected behavior in certain cases.

In [ ]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

In [ ]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

max_seq_length = 3072 # max sequence length for model and packing of the dataset

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
#model.to(device)


trainer = SFTTrainer(
    model=original_model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    #device_map={"": device},
    #formatting_func=formatting_func,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

# Model training AND Saving the fine-tuned model


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(

its related to past_key_values, you can disable this warning by setting
model.config.use_cache = False, when using gradient checkpointing during training.
but during inference make sure to set it back to True.

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(

possible solution:
model.gradient_checkpointing_enable() fix it. maybe you can try @lucasjinreal    

https://github.com/huggingface/transformers/issues/28536

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py
#  1406

# save model
trainer.save_model()

In [ ]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

# Model evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
peft_model_id = "/content/gdrive/MyDrive/model/Mistral-7B-v0.1_McGill-MEDAL"


# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  load_in_8bit=True # Load the model in 8-bit precision
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)


# load into pipeline
generation_pipeline = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [7]:
# Clear the cache to release memory
import gc
gc.collect()
torch.cuda.empty_cache()

In [79]:
index=10

In [80]:
print(f"Question: {datasetF['Question'][index]} \nAnswer BY the Dataset: {datasetF['Answer'][index]}")

Question: in this paper a new MM of SOC is introduced this model called the mRNA paradigm is motivated by the problem of gene expression initiation in the newlyborn daughter cells T3 mitosis the MM is fundamentally different in dynamics and properties from the well known sandpile paradigm simulation experiments demonstrate that a critical total number of proteins exists below which transcription is impossible above this critical threshold the system enters the regime of selfsustained oscillations with standard deviations and periods proportional to the genes complexities with probability one the BL between these two regimes is very sharp importantly such a selforganization emerges without any deterministic feedback loops or external supervision and is a result of CR random redistribution of proteins between inactive genes given the size of the genome the domain of selforganized oscillatory motion is also limited by the genes maximal complexities below the critical complexity all the re

In [81]:
prompt=datasetF['Question'][index]
outputs = generation_pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [82]:
print('Question: %s'%prompt)

ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]


print()
print(f"Generated Answer:\n{ganswer}")

Question: in this paper a new MM of SOC is introduced this model called the mRNA paradigm is motivated by the problem of gene expression initiation in the newlyborn daughter cells T3 mitosis the MM is fundamentally different in dynamics and properties from the well known sandpile paradigm simulation experiments demonstrate that a critical total number of proteins exists below which transcription is impossible above this critical threshold the system enters the regime of selfsustained oscillations with standard deviations and periods proportional to the genes complexities with probability one the BL between these two regimes is very sharp importantly such a selforganization emerges without any deterministic feedback loops or external supervision and is a result of CR random redistribution of proteins between inactive genes given the size of the genome the domain of selforganized oscillatory motion is also limited by the genes maximal complexities below the critical complexity all the re